Gm(h) -> γκολ μιας ομαδας που επαιζε εντος εδρας

Gm(a) -> γκολ μιας ομαδας που επαιζε εκτος εδρας

Τ = {t1, t2, ..., tk} -> κ πληθους ομαδες 

Μ = {m1, m2, ..., mj} -> j πληθους αγωνες

r μια συναρτηση που το αποτελεσμα της ανηκει στο συνολο {H, D, A} -> αποτελεσμα αγωνα
r(match, home_team, away_team) = {
    return H if DGm(home_team, away_team) > 0
    return D if DGm(home_team, away_team) = 0
    return A if DGm(home_team, away_team) < 0
}

DGm(home_team, away_team) = Gm(h) - Gm(a)

R2 -> (x, y)
R8 -> (x1, x2, x3, ... x8)
Rn -> (x1, x2, x3, ..., xn)
f(t) ανηκει στο συνολο R8 και η μεταβλητη  t ειναι μια ομαδα απο το συνολο T

k ανηκει στο συνολο {Β365, BW, IW, LB} -> εταιριες

ψ(match) -> αποδοσεις για καθε πιθανο αποτελεσμα απο τις κ εταιριες (ειναι ενα τετραδα της μορφης: (a, b, c, d))
 
 
 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 as sq

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

% Create a connection to the database.
% Retrieve the contents of the Team_Attributes table containing the
% following columns: id,
% Retrieve the contents of the Match table containing the following
% columns: id, B365H, B365D, B365A,BWH, BWD, BWA, IWH, IWD, IWA, LBH, LBD, LBA
and collumns needed to calculate game result and teams ids

In [ ]:
#just reading data
con = sq.connect("../input/database.sqlite")
team_att = pd.read_sql_query("SELECT * from Team_Attributes", con)
team_att = team_att [['id','team_api_id','buildUpPlaySpeed','buildUpPlayPassing','chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth']]
team = pd.read_sql_query("SELECT * from Team", con)
match = pd.read_sql_query("SELECT * from Match", con)
match = match[['id','match_api_id','home_team_api_id', 'home_team_goal', 'away_team_goal','away_team_api_id','B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD',
              'LBA']]

show match

In [ ]:
match

#clean match from empties

In [ ]:
match.dropna()

add columns home_win, away_win, draw με 0 ή 1 με βαση ποια ομαδα εβαλε τα περισσότερα goals

In [ ]:
match.loc[(match['home_team_goal'] > match['away_team_goal']),'home_win']='1'
match.loc[(match['home_team_goal'] < match['away_team_goal']),'away_win']='1'
match.loc[(match['home_team_goal'] == match['away_team_goal']),'draw']='1'

In [ ]:
# replace NaN with 0
match.fillna(0, inplace=True)
match

Mετατροπη των αποδόσεων σε προτεινόμενες πιθανότητες (implied probabilities)

In [ ]:
team_att

In [ ]:
#merge match and team_att 

In [ ]:
#shuffle match rows so split tables are randomized
#match = match.reindex(np.random.permutation(match.index))

#split match data into training, validation, and test sets
m_train = match.iloc[:17861]
m_valid = match.iloc[17861:21108]
m_test = match.iloc[21108:]